In [1]:
import numpy as np 
import pandas as pd
import re

In [2]:
reviews = pd.read_csv('amazon_baby.csv')

In [3]:
reviews = reviews.dropna()
reviews['review'] = reviews['review'].str.lower()
reviews['review'] = reviews['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

text_data = reviews['review'].tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)

2023-05-20 16:47:55.368774: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 16:47:55.979646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
sequences = tokenizer.texts_to_sequences(text_data)

[[43,
  2531,
  453,
  22,
  626,
  18,
  12,
  11,
  1109,
  21,
  218,
  675,
  4,
  65,
  282,
  51829,
  6912,
  380,
  51830,
  35484,
  51831,
  22,
  538,
  42,
  6,
  1688,
  1096,
  1680,
  2,
  34,
  565,
  984,
  151,
  4,
  38,
  380,
  453,
  9,
  347,
  2,
  2749,
  2,
  17,
  107,
  51832,
  269,
  383,
  6690,
  453,
  51833,
  67,
  482,
  85,
  79,
  2,
  148,
  5,
  707,
  47,
  62,
  27,
  22,
  983,
  5,
  55,
  1280,
  2,
  17,
  42,
  3046,
  518,
  9,
  6,
  124,
  13,
  8013,
  73,
  362,
  727],
 [3,
  252,
  1076,
  2,
  16,
  21,
  438,
  4,
  52,
  3431,
  2387,
  364,
  2,
  79,
  11,
  770,
  695,
  3,
  51834,
  11,
  5394,
  453,
  4043,
  2,
  109,
  21,
  456,
  262,
  101,
  3],
 [28,
  153,
  2,
  205,
  2,
  828,
  72,
  3,
  51835,
  1,
  500,
  145,
  261,
  51836,
  101,
  5,
  406,
  216,
  9,
  8,
  754,
  10,
  2875],
 [8,
  7,
  6,
  66,
  59,
  218,
  1,
  233,
  4,
  17,
  21,
  220,
  326,
  501,
  40,
  8,
  2,
  3,
  7,
  6,
  1591,
  5

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_seq_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length)


In [7]:
from sklearn.model_selection import train_test_split

X = padded_sequences
y = reviews['rating'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_seq_length))
model.add(LSTM(100))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adam')


2023-05-20 16:48:10.277310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-20 16:48:10.315424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-20 16:48:10.315603: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

2023-05-20 16:48:11.363165: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1575211972 exceeds 10% of free system memory.


Epoch 1/10


2023-05-20 16:48:11.887950: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-20 16:48:11.889342: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-20 16:48:11.890427: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1140/1140 [==============================] - ETA: 0s - loss: 1.0742

2023-05-20 16:53:02.556979: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-20 16:53:02.558460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-20 16:53:02.559607: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1140/1140 [==============================] - 311s 270ms/step - loss: 1.0742 - val_loss: 0.6440
Epoch 2/10
1140/1140 [==============================] - 283s 248ms/step - loss: 0.5375 - val_loss: 0.5678
Epoch 3/10
1140/1140 [==============================] - ETA: 0s - loss: 0.4006